In [13]:
import geopandas as gpd
import pandas as pd
import boto3
import os

In [14]:
bucket_name = 'opera-calval-database-dswx'
private_bucket_name = 'opera-calval-database-dswx-private'

In [21]:
local_planet_dir = './downloads/crop/'

In [27]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

In [23]:
collocation = gpd.read_file('Collocation_HLS_Planet_2021.geojson')


In [24]:
#test = collocation.head(100)
#for row in test.iterrows():
#    print(row[1].pl_ID)

In [25]:
fileList = os.listdir(local_planet_dir)
print(fileList)

['20210924_000522_94_2421_3B_AnalyticMS_SR_8b.tif']


In [29]:
#fileList = fileList[0:1]
#print(fileList)
for file in fileList:
    if file[-3:] != 'tif':
        continue
    
    ID = file.split('_3B_Analytic')[0]
    print('Adding Image ID: '+ID)
    print('\tBuilding Image Table Entry')
    search = collocation[(collocation.pl_ID == ID)].sort_values('hls_pl_AOI_intersect',ascending=False)
    search = search.iloc[[0]]
    site = search.chip_ID.iloc[0]
    s3_key_image = 'data/'+site+'/'+ID+'/'+file
    new_image = search.drop(columns=['hls_Date','hls_Cloud_Cover','hls_Footprint','hls_AOI_intersect','hls_pl_AOI_intersect','chip_rID'])
    new_image = new_image.rename(columns = {'hls_ID':'collocated_dswx','pl_Date':'datetime','pl_ID':'image_name','pl_Cloud_Cover':'cloud_cover','pl_AOI_intersect':'site_coverage','chip_ID':'site_name'})
    new_image['s3_bucket'] = private_bucket_name
    new_image['s3_key_image'] = s3_key_image
    #print(new_image.head())
    obj = s3.Object(bucket_name,'image.geojson')
    image_data = obj.get()['Body']
    image_gdf = gpd.read_file(image_data)
    #print(image_gdf)
    if 'image_name' in image_gdf.columns:
        if len(image_gdf[image_gdf.image_name==new_image.image_name.iloc[0]]) != 0:
            print('\timage_name: '+new_image.image_name.iloc[0]+' is already in image table')
            #continue
    
    print('\tUploading File')
    result = s3_client.list_objects_v2(Bucket=private_bucket_name, Prefix=s3_key_image)
    
    if 'Contents' in result:
        print("\tImage file already exists in bucket")
    else:
        response = s3_client.upload_file(local_planet_dir+file,private_bucket_name,s3_key_image)
        #print('\tUpload manually skipped')
    
    print('\tUpdating Image Table')
    
    image_gdf_upd = image_gdf.append(new_image,ignore_index=True)
    image_upd_bytes = bytes(image_gdf_upd.to_json(drop_id=True).encode('UTF-8'))
    s3object = s3.Object(bucket_name,'image.geojson')
    s3object.put(Body=image_upd_bytes)
    print('\tCompleted')

Adding Image ID: 20210924_000522_94_2421
	Building Image Table Entry
	image_name: 20210924_000522_94_2421 is already in image table
	Uploading File
	Updating Image Table


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


	Completed


In [20]:
obj = s3.Object(bucket_name,'image.geojson')
image_data = obj.get()['Body']
image_gdf = gpd.read_file(image_data)
print(image_gdf)

     cloud_cover                     collocated_dswx             datetime  \
0            0.0  HLS.S30.T10VEK.2021268T192141.v2.0  2021-09-25T19:17:08   
1            0.0  HLS.L30.T18UXG.2021300T154817.v2.0  2021-10-27T15:14:45   
2            0.0  HLS.S30.T18FYH.2021302T141741.v2.0  2021-10-30T14:26:13   
3            0.0  HLS.S30.T36LWH.2021268T073641.v2.0  2021-09-26T07:24:14   
4            0.0  HLS.S30.T15VUK.2021244T175911.v2.0  2021-09-01T17:52:05   
..           ...                                 ...                  ...   
125          0.0  HLS.S30.T35SMV.2021267T085709.v2.0  2021-09-24T08:12:39   
126          0.0  HLS.S30.T37SBT.2021270T081721.v2.0  2021-09-28T07:28:02   
127          0.0  HLS.S30.T20KMB.2021267T141739.v2.0  2021-09-24T13:34:00   
128          0.0  HLS.L30.T33JYG.2021292T084713.v2.0  2021-10-18T08:55:12   
129          0.0  HLS.S30.T50HNG.2021248T020439.v2.0  2021-09-06T01:23:32   

                  image_name instrument     provider  resolution  \
0    20

In [27]:
print(new_image.image_name.iloc[0])
if len(image_gdf[image_gdf.image_name==new_image.image_name.iloc[0]]) != 0:
    print('\timage_name: '+new_image.image_name.iloc[0]+' is already in image table')


20210925_191708_94_2408
	image_name: 20210925_191708_94_2408 is already in image table
